In [16]:
version = 4.0


import os
import pandas as pd
import shutil
input_dir = "Raw Data Files"
print(f'***----🎀  TIP Data Software v{version}  🎀----***')
print('***-----This program is created by ASF Technical Branch AIIAP Lahore-----***')
print('***----For any assistance or error in the program Please contact Us at 03000705208 ----***')
print(f'***----This is version {version} of TIP Data Software.  ----***')
print(
    """
--------------------------------------------------------------------------------------------------------------------------
            ***Version 4.0 Updates***                                                                                       
Update includes:
-> Result calculation method changed. Now weihtage avarage is being used 
    (and normalized the Bags Count data with the max of Bags Count):
    Weightage of Tip Count = 80%
    Weightage of Bags Scanned = 20%
    
            ***Version 3.0 Updates***                                                                                       
Update includes:
-> All files can be placed in one folder. Automatic separation of diffferent files based on data and date.
-> Process all files from any date range.

            ***Version 2.0 Updates***   
Update includes:
-> Data types of columns corrected. Some digits data points were treated as string, converted them to integer 
'LOGIN ID', 'BAG COUNT', 'TIP COUNT', 'HIT'
-> Output as Excel instaed of CSV
--------------------------------------------------------------------------------------------------------------------------
"""
)
print(f'Please place all your relevent TIP data files  in the folder  "{input_dir}"')
qq = input('Have you followed above instruction? If Yes press "Y"......')



def create_directory(directory_path):
  """Creates a directory at the specified path.
  Args:
    directory_path: The path to the directory to create.
  """

  try:
    os.makedirs(directory_path, exist_ok=True)
    print(f"Directory '{directory_path}' created successfully.")
  except OSError as error:
    print(f"Error creating directory: {error}")
all_data_files_folder = "All Data Files"
create_directory(all_data_files_folder)

# Scanning "input_dir" folder and getting all files paths in subdirs into files_list.
files_list=[]
def recursive_scandir(directory):
    for entry in os.scandir(directory):
        if entry.is_dir():
            recursive_scandir(entry.path)
        else:
            if '.txt' in entry.path:
                files_list.append(entry.path)  # Or perform other actions with the file

recursive_scandir(input_dir)
# Copying files form input_dir to "All Data Files" dir
for file in files_list:
    shutil.copy2(file, all_data_files_folder)

# In Smith Detection Machines all files ends with a specific number 1-4 according to data, like tip data files end with 3. We can classify files:
classification_dict={}
for file in files_list:
# def classification(file):
    file_name= file.split("\\")[-1].replace('.txt','')
    file_serial=file_name[-1]
    date=file_name[-7:-1]
    if file_serial not in classification_dict:
        classification_dict[file_serial]={}
    if date not in classification_dict[file_serial]:
        classification_dict[file_serial][date]=[]
    if date in classification_dict[file_serial]:
        classification_dict[file_serial][date].append(file_name)

if qq == 'Y' or qq == 'y':
    
    df = pd.DataFrame()

    for date in classification_dict['3']:
        for filess in classification_dict['3'][date]:
            df1 = pd.read_csv(f'{all_data_files_folder}//{filess}.txt', delimiter='\t')
            df = pd.concat([df, df1], ignore_index=True)
        df = df[['NAME', 'LOGIN ID', 'BAG COUNT', 'TIP COUNT', 'HIT']]
        df = df[df['NAME'] != 'SERVICE']
        df = df[df['LOGIN ID']!='unknown user'].dropna()
        df = df.astype({
        'NAME': str,
        'LOGIN ID': int,
        'BAG COUNT': int,
        'TIP COUNT': int,
        'HIT': int
    })
        df = df.groupby(['LOGIN ID'], as_index=False).agg({
        'NAME': 'first',
        'BAG COUNT': 'sum',
        'TIP COUNT': 'sum',
        'HIT': 'sum'
    })
        df = df[['NAME', 'LOGIN ID', 'BAG COUNT', 'TIP COUNT', 'HIT']]
        bags_count_max = df['BAG COUNT'].max()
        bags_count_weightage= 20
        tip_count_weightage = 100-bags_count_weightage
        # In below result calculation bags_count_weightage"% weightage to Baggs Count and 70% weightage to Tip Count
        df['RESULT'] =((df['BAG COUNT']/bags_count_max)*bags_count_weightage+(df['HIT'] / df['TIP COUNT'])*tip_count_weightage).round(0) #(df['HIT'] / df['TIP COUNT'] * 100).round(0)
        def grades(x):
            if x == 100:
                return 'A+'
            if x >= 90:
                return 'A'
            if x >= 80:
                return 'B'
            if x >= 70:
                return 'C'
            return 'D'
        df['GRADE'] = df['RESULT'].apply(grades)
        df3 = df.dropna(subset=['RESULT'])
        df3 = df3.sort_values(by=['RESULT'], axis=0, ascending=False)
        df3=df3[df3['BAG COUNT']!=0]
        df3.insert(0, 'SER NO', range(1, len(df3) + 1))
        df3.to_excel(f'Tip Data {date[0:2]}-{date[2:]}.xlsx',index=False)
    print('Programme completed successfully.')


***----🎀  TIP Data Software v4.0  🎀----***
***-----This program is created by ASF Equipment AIIAP Lahore-----***
***----For any assistance or error in the program Please contact Us at 03000705208 ----***
***----This is version 4.0 of TIP Data Software.  ----***

--------------------------------------------------------------------------------------------------------------------------
            ***Version 4.0 Updates***                                                                                       
Update includes:
-> Result calculation method chaged. Now wiehtage avarage is being used:
-> Process all files from any date range.

            ***Version 3.0 Updates***                                                                                       
Update includes:
-> All files can be placed in one folder. Automatic separation of diffferent files based on data and date.
-> Process all files from any date range.

            ***Version 2.0 Updates***   
Update includes:
-> Dat

In [3]:
bags_count_max

np.int64(19898)

In [6]:
(df['BAG COUNT']/bags_count_max+df['HIT'] / df['TIP COUNT'] * 100).round(0)

0      50.0
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
144    68.0
145    93.0
146     NaN
147     NaN
148     NaN
Length: 149, dtype: float64

In [5]:
((df['BAG COUNT']/bags_count_max+df['HIT'] / df['TIP COUNT']) * 100).round(0)

0      50.0
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
144    91.0
145    95.0
146     NaN
147     NaN
148     NaN
Length: 149, dtype: float64

In [8]:
df

,NAME,LOGIN ID,BAG COUNT,TIP COUNT,HIT
0,EQU,8,32,2,1
1,9,9,0,0,0
2,31,31,0,0,0
3,1111,1111,0,0,0
4,1276,1276,0,0,0
...,...,...,...,...,...
144,SHUMAILA_ZAFAR,18921,4646,424,288
145,AHMAD_RAZA,68425,319,29,27
146,90003,90003,0,0,0
147,119144,119144,0,0,0
